# Generation: Generating a Response

In [1]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
pip show langchain

Name: langchain
Version: 1.2.3
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Premasis\Development\Pycharm\LangChainSample\.venv\Lib\site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [4]:
vectorstore = Chroma(persist_directory = "./local-database",
                     embedding_function = OpenAIEmbeddings(model='text-embedding-ada-002'))

In [5]:
len(vectorstore.get()['documents'])

40

In [6]:
retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [7]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [9]:
chat = ChatOpenAI(
    model="gpt-4",
    seed=365,
    max_tokens=250,
)


In [11]:
question = "What software do data scientists use?"

In [12]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template 
         | chat 
         | StrOutputParser())

In [13]:
chain.invoke(question)

'Data scientists utilize a variety of software. Two of the most popular tools across all categories are R and Python. These programming languages boast a significant advantage as they can manipulate data and are integrated within multiple data and data science software platforms, making them suitable not just for mathematical and statistical computations, but for solving a wide array of business and data-related problems. Another significant software framework utilized in data science is Hadoop, it is designed to address issues related to the complexity and computational intensity of big data by distributing computational tasks across multiple computers. For business intelligence visualizations, data scientists often use software such as Power BI, SaS, Qlik, and Tableau. \n\nResources: *Programming Languages & Software Employed in Data Science - All the Tools You Need*'

In [13]:
print('Data scientists use a variety of software tools. R and Python are the two most popular tools as they can manipulate data and are integrated within multiple data and data science software platforms. They are adaptable and can solve a wide range of business and data-related problems. Hadoop is a software framework designed to handle the complexity and computational intensity of big data by distributing computational tasks on multiple computers. Additionally, Power BI, SaS, Qlik, and Tableau are top-notch examples of software designed for business intelligence visualizations.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need')

Data scientists use a variety of software tools. R and Python are the two most popular tools as they can manipulate data and are integrated within multiple data and data science software platforms. They are adaptable and can solve a wide range of business and data-related problems. Hadoop is a software framework designed to handle the complexity and computational intensity of big data by distributing computational tasks on multiple computers. Additionally, Power BI, SaS, Qlik, and Tableau are top-notch examples of software designed for business intelligence visualizations.

Resources: Programming Languages & Software Employed in Data Science - All the Tools You Need
